In [12]:
import pandas as pd
import numpy as np
import os
import sys
import Mixture
import multiprocessing
from joblib import Parallel, delayed
from random import seed
from random import sample 
from random import randint

#Library to read RDS format
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

In [13]:
lines = [2, 6]
cpu = multiprocessing.cpu_count()
rango = 1000

In [19]:
dataFrameSignature = pd.read_csv('data/yourCSV.csv') 

In [20]:
def createBetas (X, a, b, n):

    ns = randint(a, b)
    r = np.random.uniform(1, 0.2, ns)
    rn = r/r.sum()
    id = sample(range(n-1), ns)
    betas = np.repeat(.0, n, axis=0)
    for index, i in enumerate(id, start = 0):
        betas[i] = rn[index]

    vector = X.to_numpy(copy=True)    
    vector = vector.flatten()
        
    A = np.dot(X, betas) + vector[sample(range(len(vector)), len(X.index))]

    return betas, id, A

In [21]:
X = dataFrameSignature.iloc[:, 1:]

subjects = Parallel(n_jobs=cpu, backend='threading')(delayed(createBetas)(X = X, a = lines[0], b = lines[1], n = len(X.columns)) for i in range(rango))

# Getting expression matrix
vector = [x[2] for x in subjects]
columns = ['V' + str(x+1) for x in range(len(subjects))]
Y2 = pd.DataFrame(np.column_stack(vector), columns=columns)
Y2.insert(0, 'Gene symbol', dataFrameSignature['Gene symbol'])

# Getting Real Betas Matrix
vector = [x[0] for x in subjects]
columns = ['V' + str(x+1) for x in range(len(subjects))]
betas = pd.DataFrame(np.column_stack(vector), columns=columns)

X = dataFrameSignature

vector = [len(x[1]) for x in subjects]
ids = pd.DataFrame(np.column_stack(vector))   

In [23]:
betas

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V991,V992,V993,V994,V995,V996,V997,V998,V999,V1000
0,0.000000,0.502811,0.454752,0.167258,0.000000,0.215948,0.535272,0.145751,0.000000,0.264716,...,0.376840,0.252751,0.155439,0.000000,0.000000,0.000000,0.000000,0.175348,0.000000,0.146273
1,0.000000,0.000000,0.000000,0.129230,0.183752,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.227507,0.296282,0.000000,0.109081,0.000000,0.304872,0.000000,0.082980,0.353891,0.073859
2,0.000000,0.119712,0.336865,0.157087,0.000000,0.000000,0.000000,0.234765,0.000000,0.072633,...,0.000000,0.079687,0.000000,0.216050,0.111002,0.000000,0.000000,0.225321,0.000000,0.263963
3,0.000000,0.377477,0.000000,0.071418,0.263163,0.000000,0.000000,0.228725,0.273737,0.115481,...,0.000000,0.126537,0.364723,0.000000,0.349242,0.225838,0.264335,0.092221,0.000000,0.092616
4,0.000000,0.000000,0.000000,0.315524,0.000000,0.237616,0.000000,0.000000,0.000000,0.158002,...,0.163713,0.000000,0.278139,0.000000,0.147236,0.250998,0.000000,0.000000,0.168081,0.250045
5,0.436054,0.000000,0.000000,0.000000,0.199695,0.000000,0.000000,0.263115,0.256863,0.279624,...,0.136043,0.071194,0.000000,0.232688,0.123061,0.000000,0.735665,0.197026,0.478028,0.000000
6,0.000000,0.000000,0.208382,0.000000,0.119498,0.394620,0.464728,0.127643,0.206366,0.109544,...,0.095896,0.000000,0.000000,0.146184,0.269459,0.218292,0.000000,0.227104,0.000000,0.173244
7,0.563946,0.000000,0.000000,0.159484,0.233892,0.151817,0.000000,0.000000,0.263035,0.000000,...,0.000000,0.173549,0.201700,0.295997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
Y2

,Gene symbol,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V991,V992,V993,V994,V995,V996,V997,V998,V999,V1000
0,ARID5B,33.237687,70.004221,1056.257283,75.998576,606.757172,1079.703404,1099.903693,520.480681,539.262485,...,995.897857,810.997562,82.125528,498.925694,693.335024,1559.405389,42.868952,558.271228,26.015060,439.750067
1,ZNF460,26.172333,43.537807,223.402609,30.690271,151.602554,646.409374,467.474151,790.127014,3380.805424,...,274.335580,489.639168,41.772076,298.386691,483.863220,230.133870,179.894142,328.406265,232.817597,2019.223556
2,RBM47,109.314441,736.350077,136.280086,269.460696,101.129499,84.368798,92.458961,488.437630,302.876185,...,71.700676,1202.688853,142.731337,160.251038,143.508774,452.588464,104.504425,2417.902865,170.170553,6484.288262
3,GRN,199.699501,2899.913194,2775.966406,3179.817075,1603.252006,5487.312874,3288.044927,3441.053205,1972.853978,...,958.627424,1378.194111,4269.217560,1839.559826,3513.076996,3145.766079,2488.649360,2433.423885,1546.477014,3929.304613
4,CTSB,2060.305599,2641.821624,3824.150966,3378.427134,2719.162696,1295.794071,2070.942599,3123.098227,1164.683130,...,910.904406,1390.583251,2874.623739,2222.495951,7747.885804,3693.747531,1756.121910,3344.505323,963.271179,4113.015548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12360,OR2L2,30.598337,29.551740,102.428036,69.746195,554.363247,26.172729,241.501941,28.244258,29.620185,...,545.311209,57.132587,1263.869664,206.059937,72.882510,1236.484502,213.956954,25.273767,49.828179,36.304298
12361,GCSAML,1614.770854,283.851839,32.876204,156.151444,3359.295503,54.614246,33.286202,59.328050,22.607001,...,22.134863,1047.459657,685.846008,78.910881,74.949702,53.160030,218.975359,88.971986,282.284001,87.941525
12362,TMEM231,49.544998,74.324778,172.082769,43.096872,31.021561,93.263458,2533.191697,96.542658,18.758440,...,91.690576,24.249163,21.062645,40.048350,1135.347276,75.620229,189.995349,65.939426,154.982048,265.342772
12363,ZNF155,76.838942,1313.958559,101.254863,72.206799,114.216649,63.563951,165.527547,235.735576,287.283313,...,133.912598,93.175893,154.573671,139.223580,89.126343,63.836629,82.270422,53.104712,54.417702,46.981812


In [ ]:
#Generate xlsx
with pd.ExcelWriter('betas.list.TIL9.xlsx') as writer:
    betas.to_excel(writer, sheet_name='beta')
    ids.to_excel(writer, sheet_name='id')
    Y2.to_excel(writer, sheet_name='A')
